# Using ReliableTool to Generate Sub-Questions (Synchronous Version)
This notebook demonstrates how to use the `ReliableTool` synchronously.

In [ ]:
import asyncio
import logging
import os
from typing import Any, Optional

# Import necessary components from autogen
from autogen import LLMConfig, config_list_from_json
from autogen.tools.experimental.google_search import GoogleSearchTool
from autogen.tools.experimental.reliable import ReliableTool, ReliableToolError

# Configure logging
# Set level to DEBUG to see more internal details if needed
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s [%(levelname)s] %(name)s: %(message)s')
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(name)s: %(message)s")
logger = logging.getLogger(__name__)

print("Successfully imported components from autogen.")


## 1. Define the Google Search Tool


In [ ]:
search_api_key = os.getenv("GOOGLE_SEARCH_API_KEY")
search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")


gs_tool: Optional[GoogleSearchTool] = None
if search_api_key and search_engine_id:
    try:
        gs_tool = GoogleSearchTool(
            search_api_key=search_api_key,
            search_engine_id=search_engine_id,
        )
        logger.info("GoogleSearchTool instance created successfully.")
    except Exception as e_gs_init:
        logger.error(f"Failed to initialize GoogleSearchTool: {e_gs_init}", exc_info=True)
else:
    logger.warning("GoogleSearchTool cannot be initialized due to missing API keys.")

## 2. Configure LLM and ReliableTool

In [ ]:
llm_config: Optional[LLMConfig] = None
try:
    config_list = config_list_from_json(
        "OAI_CONFIG_LIST",
    )
    llm_config = LLMConfig(
        config_list=config_list,
        temperature=0.7,
    )
    print(f"Using LLM config: {config_list[0].get('base_url', 'Default Endpoint')}, Model: {config_list[0]['model']}")
except Exception as e_config:
    print(f"Error creating LLM config: {e_config}. Tools cannot be initialized.")
    llm_config = None

runner_system_message = """You are an assistant that prepares search queries for the Google Search tool.
Your goal is to take a user request (the 'task') and formulate the most effective search query string for the tool to execute.
The internal tool to call is named 'execute_google_search'.
You MUST provide the search query as the 'query' argument to this tool.
You MUST also provide a 'hypothesis' argument summarizing the type of information or answer expected from the search results (e.g., "a list of recent news articles", "a definition", "the official website").

Example:
User Task: "Find recent news about the AutoGen library."
Your Tool Call should be: execute_google_search(query="AutoGen library recent news", hypothesis="A list of recent news articles or blog posts about the AutoGen library.")

Example:
User Task: "What is the capital of France?"
Your Tool Call should be: execute_google_search(query="capital of France", hypothesis="The name of the capital city of France.")

You MUST invoke the 'execute_google_search' tool exactly once per response using a tool call format. Do not add conversational text.
"""

validator_system_message = "Make sure all search results are specifically from the link github.com."

reliable_search_tool: Optional[ReliableTool] = None
if llm_config and gs_tool:
    try:
        # Pass the GoogleSearchTool *instance* as func_or_tool
        reliable_search_tool = ReliableTool(
            name="ReliableGoogleSearch",
            func_or_tool=gs_tool,  # Pass the initialized GoogleSearchTool
            description="Reliably performs a Google search based on a task description and validates the relevance of the results.",
            runner_llm_config=llm_config,
            validator_llm_config=llm_config,
            runner_system_message=runner_system_message,
            max_retries=4,
        )
        logger.info("ReliableTool instance for Google Search created successfully.")
    except Exception as e_reliable_init:
        logger.error(f"Error creating ReliableTool for Google Search: {e_reliable_init}", exc_info=True)
else:
    logger.warning("LLM Configuration or GoogleSearchTool instance not ready. Cannot create ReliableTool.")

## 3. Get User Input and Run the Tool Synchronously

In [ ]:
async def run_reliable_search_task():
    """Defines a search task and runs the ReliableTool asynchronously."""
    if not reliable_search_tool:
        logger.error("ReliableTool for search was not initialized. Exiting.")
        return

    search_task = "What is the latest news about the AutoGen project?"
    validation_criteria = None

    logger.info(f"\nAttempting to reliably perform search task: '{search_task}'")
    print("-" * 40)

    try:
        # Run the ReliableTool asynchronously using the .a_run() method
        # It expects the 'task' argument as defined in its tool_schema
        result: Optional[Any] = await reliable_search_tool.a_run(
            task=search_task,
            validation_prompt_addition=validation_criteria,  # Pass None if not used or disabled
            ground_truth=["The user wants to know specifically if AG2 was mentioned on reddit"],
        )

        print("-" * 40)
        # GoogleSearchTool typically returns a formatted string of search results
        if isinstance(result, str):
            logger.info("\n✅ Successfully performed search task:")
            print("\n--- Search Results ---")
            print(result)
            print("--- End Search Results ---")
        elif result is not None:
            logger.info("\n✅ Successfully performed search task (result is not a string):")
            print(f"   Result Type: {type(result)}")
            print(f"   Result: {result}")
        else:
            # This might happen if the underlying tool successfully returns None
            logger.warning("\n⚠️ Tool execution finished successfully, but the result is None.")

    except ReliableToolError as e:
        print("-" * 40)
        attempt_count = e.final_context.attempt_count if e.final_context else "N/A"
        logger.error(f"\n❌ ReliableTool failed search task after {attempt_count} attempt(s).")
        print(f"Error Summary: {e}")  # Contains failure summary from the tool
        # Optionally print final context for debugging
        if e.final_context:
            print("\n--- Final Context (Debug) ---")
            try:
                # Use model_dump_json for Pydantic v2
                print(e.final_context.model_dump_json(indent=2))
            except Exception as dump_err:
                print(f"(Could not dump context: {dump_err}) -> Context object: {e.final_context}")
            print("--- End Final Context ---")

    except Exception as e_unexpected:
        print("-" * 40)
        logger.critical(f"\n❌ An unexpected error occurred during execution: {e_unexpected}", exc_info=True)


asyncio.run(run_reliable_search_task())
print("\nScript finished.")